# Activity Log routes

> routes for interacting with the activity log

In [ ]:
#| default_exp routes.activity_log

In [ ]:
#| exporti
from pprint import pprint

from enum import Enum, auto
from typing import Optional

import aiohttp

import domolibrary.client.get_data as gd

import domolibrary.DomoAuth as dmda
import domolibrary.client.ResponseGetData as rgd


In [ ]:
async def get_activity_log_object_types( auth : dmda.DomoAuth):
    """retrieves a list of valid objectTypes to pass to the activity_log function"""
    
    url = 'https://domo-dojo.domo.com/api/audit/v1/user-audits/objectTypes'

    return await gd.get_data(
        url = url,
        method = 'GET',
        auth = auth
    )

#### sample implementation of get_activity_log_object_types

In [ ]:
import os
import pandas as pd 

try:
    token_auth = dmda.DomoTokenAuth(domo_access_token=os.environ['DOMO_DOJO_ACCESS_TOKEN'],
                                    domo_instance="domo-dojo")

    al_objecttype_res = await get_activity_log_object_types( auth = token_auth)

    print(pd.DataFrame(al_objecttype_res.response))


    ## use to update `ActivityLog_ObjectType` enum
    # [ f"{objecttype.get('translation').upper().replace(' ', '_')} = '{objecttype.get('type')}'" for objecttype in  al_objecttype_res.response]

except Exception as e:
    print(e)

                  type         translation
0         ACCESS_TOKEN        Access token
1              ACCOUNT             Account
2         ACTIVITY_LOG        Activity log
3                ALERT               Alert
4                  APP                 App
..                 ...                 ...
56                VIEW                View
57        VIRTUAL_USER        Virtual user
58     Workbench_AGENT     Workbench agent
59       Workbench_JOB       Workbench job
60  Workbench_SCHEDULE  Workbench schedule

[61 rows x 2 columns]


In [ ]:
#| export
class ActivityLog_ObjectType(Enum):
    ACCESS_TOKEN = 'ACCESS_TOKEN'
    ACCOUNT = 'ACCOUNT'
    ACTIVITY_LOG = 'ACTIVITY_LOG'
    ALERT = 'ALERT'
    APP = 'APP'
    APPDB_COLLECTION = 'MAGNUM_COLLECTION'
    APPDB_DATASTORE = 'MAGNUM_DATASTORE'
    AUTHORITY = 'AUTHORITY'
    BEAST_MODE_FORMULA = 'BEAST_MODE_FORMULA'
    BUZZ_CHANNEL = 'CHANNEL'
    BUZZ_GROUP_CHAT = 'GROUP_CHAT'
    BUZZ_THREAD = 'HUDDLE'
    CARD = 'CARD'
    CHART_COLOR_PALETTE = 'CHART_COLOR_PALETTE'
    COLLECTION = 'COLLECTION'
    CUSTOMER = 'CUSTOMER'
    CUSTOMER_STATE = 'CUSTOMER_STATE'
    CUSTOMER_TIER = 'CUSTOMER_TIER'
    DATA_SCIENCE_NOTEBOOK = 'DATA_SCIENCE_NOTEBOOK'
    DATAFLOW = 'DATAFLOW_TYPE'
    DATASET = 'DATA_SOURCE'
    DATASOURCE = 'DATASOURCE'
    DEPLOYMENT = 'DEPLOYMENT'
    DRILL_VIEW = 'DRILL_VIEW'
    EASY_INVITE_LINK = 'EASY_INVITE_LINK'
    ENABLED = 'ENABLED'
    FILE = 'FILE'
    FILE_VERSION = 'FILE_REVISION'
    GROUP = 'GROUP'
    LICENSE_PAGE = 'LICENSE_PAGE'
    LOGIN_SETTINGS = 'LOGIN_SETTINGS'
    NAME = 'NAME'
    PDP_FILTER = 'ADC_FILTER'
    PDP_POLICY = 'ADC_POLICY'
    PAGE = 'PAGE'
    PAGE_ANALYZER = 'PAGE_ANALYZER'
    PAGE_COLLECTION = 'PAGE_COLLECTION'
    PROJECT = 'PROJECT'
    PROJECT_LIST = 'PROJECT_LIST'
    PROJECT_TASK = 'PROJECT_TASK'
    PROJECT_TASK_ATTACHMENT = 'PROJECT_TASK_ATTACHMENT'
    PROJECT_TASK_OWNER = 'PROJECT_TASK_OWNER'
    PROXIER_EMAIL = 'PROXIER_EMAIL'
    PUBLIC_EMBED_URL = 'PUBLIC_URL'
    PUBLICATION = 'PUBLICATION'
    REPOSITORY = 'REPOSITORY'
    REPOSITORY_AUTHORIZATION = 'REPOSITORY_AUTHORIZATION'
    ROLE = 'ROLE'
    SEGMENT = 'SEGMENT'
    SSO_PAGE = 'SSO_PAGE'
    SUBSCRIBER = 'PROXY_USER'
    USER = 'USER'
    USER_STATE = 'USER_STATE'
    VARIABLE = 'VARIABLE'
    VARIABLE_CONTROL = 'VARIABLE_CONTROL'
    NOTEBOOK_VIEW = 'CONTAINER_VIEW'
    VIEW = 'VIEW'
    VIRTUAL_USER = 'VIRTUAL_USER'
    WORKBENCH_AGENT = 'Workbench_AGENT'
    WORKBENCH_JOB = 'Workbench_JOB'
    WORKBENCH_SCHEDULE = 'Workbench_SCHEDULE'

In [ ]:
# | export
async def search_activity_log(domo_auth: dmda.DomoAuth,
                              start_time: int,  # epoch time in milliseconds
                              end_time: int,  # epoch time in milliseconds
                              maximum: int,
                              object_type: Optional[ActivityLog_ObjectType] = None,
                              session: aiohttp.ClientSession = None,
                              debug_api: bool = False,
                              debug_loop: bool = False):
    is_close_session = False

    if not session:
        session = aiohttp.ClientSession()
        is_close_session = True

    url = f'https://{domo_auth.domo_instance}.domo.com/api/audit/v1/user-audits'

    if object_type and object_type != ActivityLog_ObjectType.ACTIVITY_LOG:
        url = f"{url}/objectTypes/{object_type.value}"

    fixed_params = {
        'end': end_time,
        'start': start_time
    }

    offset_params = {
        'offset': 'offset',
        'limit': 'limit',
    }

    def arr_fn(res) -> list[dict]:
        return res.response

    res = await gd.looper(auth=domo_auth,
                          method='GET',
                          url=url,
                          arr_fn=arr_fn,
                          fixed_params=fixed_params,
                          offset_params=offset_params,
                          session=session,
                          maximum=maximum,
                          debug_loop=debug_loop,
                          debug_api=debug_api)

    if is_close_session:
        await session.close()

    return res

In [ ]:
import os
import pandas as pd 
import datetime as dt

import domolibrary.utils.convert as convert


end_datetime = dt.datetime.today()
start_datetime = end_datetime - dt.timedelta(days=100)

try:
    token_auth = dmda.DomoTokenAuth(domo_access_token=os.environ['DOMO_DOJO_ACCESS_TOKEN'],
                                domo_instance="domo-dojo")

    page_res = await search_activity_log(
        object_type=ActivityLog_ObjectType.PAGE,
        start_time=convert.convert_datetime_to_epoch_millisecond(start_datetime),
        end_time = convert.convert_datetime_to_epoch_millisecond(end_datetime),
        domo_auth = token_auth,
        maximum = 16,
        debug_loop = False
    )

    print(pd.DataFrame(page_res[10:15]))

except Exception as e:
    print(e)


     objectId                  objectName
0  1040607753                   Campaigns
1  1050465610  MetaData Table Info and QA
2  1055094721             Scott's CRM APP
3  1067418640                Form Builder
4  1080655546                 Cloud Costs


In [ ]:
import os
import pandas as pd
import datetime as dt

import domolibrary.utils.convert as convert


end_datetime = dt.datetime.today()
start_datetime = end_datetime - dt.timedelta(days=100)

full_auth = dmda.DomoFullAuth(domo_instance="domo-dojo",
                              domo_username="jae@onyxreporting.com",
                              domo_password=os.environ['DOJO_PASSWORD']
                              )
try:
    activity_log_res = await search_activity_log(
        object_type=ActivityLog_ObjectType.ACTIVITY_LOG,
        start_time=convert.convert_datetime_to_epoch_millisecond(start_datetime),
        end_time=convert.convert_datetime_to_epoch_millisecond(end_datetime),
        domo_auth=full_auth,
        maximum=6,
        debug_loop=False,
        debug_api=False
    )

    print(pd.DataFrame(activity_log_res[0:5]))

except Exception as e:
    print(e)

        userName         userId userType actorName  actorId    actionType  \
0     Jae Wilson     1893952720     USER                  0      LOGGEDIN   
1     Jae Wilson     1893952720     USER                  0      LOGGEDIN   
2     Jae Wilson     1893952720     USER                  0      LOGGEDIN   
3     Jae Wilson     1893952720     USER                  0      LOGGEDIN   
4  test@test.com  test@test.com     USER                  0  FAILED_LOGIN   

  objectName objectId objectType  \
0                                  
1                                  
2                                  
3                                  
4                                  

                                   additionalComment           time  \
0  Jae Wilson logged in. Logged in from IP addres...  1673043815935   
1  Jae Wilson logged in. Logged in from IP addres...  1673043738154   
2  Jae Wilson logged in. Logged in from IP addres...  1673043737833   
3  Jae Wilson logged in. Logged in f